In [14]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from random_forest import RandomForest
from decision_tree import DecisionTree
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from datetime import datetime
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [15]:
df = pd.read_csv('/home/dotronghiep/Documents/Datasets/Data_mining/bank.csv',  parse_dates = True)

In [16]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes


In [17]:
X_df = df.iloc[:, :-1]
y_df = df.iloc[:, -1]
X = X_df.values
y = y_df.values

In [18]:
y[y == 'yes'] = 1
y[y == 'no'] = 0

In [19]:
print(X.shape)
print(y.shape)

(11162, 16)
(11162,)


In [20]:
for i in range(X.shape[1]):
    unique_values = np.unique(X[:, i])
    print(f"feature {i}: {unique_values}")

feature 0: [18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41
 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65
 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89
 90 92 93 95]
feature 1: ['admin.' 'blue-collar' 'entrepreneur' 'housemaid' 'management' 'retired'
 'self-employed' 'services' 'student' 'technician' 'unemployed' 'unknown']
feature 2: ['divorced' 'married' 'single']
feature 3: ['primary' 'secondary' 'tertiary' 'unknown']
feature 4: ['no' 'yes']
feature 5: [-6847 -3058 -2712 ... 56831 66653 81204]
feature 6: ['no' 'yes']
feature 7: ['no' 'yes']
feature 8: ['cellular' 'telephone' 'unknown']
feature 9: [1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27
 28 29 30 31]
feature 10: ['apr' 'aug' 'dec' 'feb' 'jan' 'jul' 'jun' 'mar' 'may' 'nov' 'oct' 'sep']
feature 11: [2 3 4 ... 3253 3284 3881]
feature 12: [1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27
 28 29 30 31 32 33

In [21]:
label_encoder = LabelEncoder()
X_label = X.copy()

columns_indices_to_encode = [1, 2, 3, 4, 6, 7, 8, 10, 15]  

for column_index in columns_indices_to_encode:
    X_label[:, column_index] = label_encoder.fit_transform(X_label[:, column_index])

print(X_label[:5])

[[59 0 1 1 0 2343 1 0 2 5 8 1042 1 -1 0 3]
 [56 0 1 1 0 45 0 0 2 5 8 1467 1 -1 0 3]
 [41 9 1 1 0 1270 1 0 2 5 8 1389 1 -1 0 3]
 [55 7 1 1 0 2476 1 0 2 5 8 579 1 -1 0 3]
 [54 0 1 2 0 184 0 0 2 5 8 673 2 -1 0 3]]


In [22]:
X_onehot = X_label.copy()
columns_to_onehot = [1, 2, 3, 8, 10, 15]  

ct = ColumnTransformer(
    [('onehot', OneHotEncoder(drop='first'), columns_to_onehot)],
    remainder='passthrough'  # Giữ lại các cột không được chỉ định
)

X_onehot = ct.fit_transform(X)
print(X_onehot[:5])

[[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 1.0
  0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 59 'no' 2343
  'yes' 'no' 5 1042 1 -1 0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 1.0
  0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 56 'no' 45 'no'
  'no' 5 1467 1 -1 0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 1.0
  0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 41 'no' 1270
  'yes' 'no' 5 1389 1 -1 0]
 [0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 1.0
  0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 55 'no' 2476
  'yes' 'no' 5 579 1 -1 0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 1.0
  0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 54 'no' 184
  'no' 'no' 5 673 2 -1 0]]


In [23]:
print("X's shape:", X.shape)
print("X_label's shape:", X_label.shape)
print("X_onehot's shape:", X_onehot.shape)

X's shape: (11162, 16)
X_label's shape: (11162, 16)
X_onehot's shape: (11162, 42)


In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(y)

[1 1 1 ... 0 0 0]


In [29]:
# accuracy
acc_my_tree =list()
acc_my_forest =list()
acc_sk_tree = list()
acc_sk_forest = list()

# time to build 
build_my_tree =list()
build_my_forest =list()
build_sk_tree = list()
build_sk_forest = list()

# time to predict
test_my_tree =list()
test_my_forest =list()
test_sk_tree = list()
test_sk_forest = list()

In [30]:
def comparative(X, y, loops=1, cate=True):
    y = y.astype('int')
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    i=0
    for i in range(loops):
        print ("Lan chay: ", i)

        if not cate:
            # sklearn tree
            start = datetime.now()
            sk_tree = DecisionTreeClassifier(criterion='entropy', max_depth=5)        
            sk_tree.fit(X_train, y_train)
            end = datetime.now() - start
            build_sk_tree.append(end)
            
            start = datetime.now()
            predictions = sk_tree.predict(X_test)
            end = datetime.now() - start
            test_sk_tree.append(end)

            acc = accuracy_score(y_test, predictions)
            print("Build time sklearn tree:", build_sk_tree[i])
            print("Test time sklearn tree:", test_sk_tree[i])
            print("Accuracy sklearn tree:", acc)
            print("====================================")
            acc_sk_tree.append(acc)

            # sklearn forest
            start = datetime.now()
            sk_forest = RandomForestClassifier(n_estimators=25, max_depth=5, criterion='entropy')        
            sk_forest.fit(X_train, y_train)
            end = datetime.now() - start
            build_sk_forest.append(end)
            
            start = datetime.now()
            predictions = sk_forest.predict(X_test)
            end = datetime.now() - start
            test_sk_forest.append(end)

            acc = accuracy_score(y_test, predictions)
            print("Build time sklearn forest:", build_sk_forest[i])
            print("Test time sklearn forest:", test_sk_forest[i])
            print("Accuracy sklearn forest:", acc)
            print("====================================")
            acc_sk_forest.append(acc)

        # my tree
        start = datetime.now()
        my_tree = DecisionTree()        
        my_tree.fit(X_train, y_train)
        end = datetime.now() - start
        build_my_tree.append(end)
        
        start = datetime.now()
        predictions = my_tree.predict(X_test)
        end = datetime.now() - start
        test_my_tree.append(end)

        acc = accuracy_score(y_test, predictions)
        print("Build time my tree:", build_my_tree[i])
        print("Test time my tree:", test_my_tree[i])
        print("Accuracy my tree:", acc)
        print("====================================")
        acc_my_tree.append(acc)

        # my forest
        start = datetime.now()
        my_forest = RandomForest()        
        my_forest.fit(X_train, y_train)
        end = datetime.now() - start
        build_my_forest.append(end)
        
        start = datetime.now()
        predictions = my_forest.predict(X_test)
        end = datetime.now() - start
        test_my_forest.append(end)

        acc = accuracy_score(y_test, predictions)
        print("Build time my forest:", build_my_forest[i])
        print("Test time my forest:", test_my_forest[i])
        print("Accuracy my forest:", acc)
        print("====================================")
        acc_my_forest.append(acc)


In [31]:
comparative(X_label, y, 1, cate=False)

Lan chay:  0
Build time sklearn tree: 0:00:00.042367
Test time sklearn tree: 0:00:00.001185
Accuracy sklearn tree: 0.7698163905060457
Build time sklearn forest: 0:00:00.757703
Test time sklearn forest: 0:00:00.022698
Accuracy sklearn forest: 0.8343036274070756
Build time my tree: 0:00:54.499312
Test time my tree: 0:00:00.002250
Accuracy my tree: 0.7922077922077922


In [ ]:
    #names =('decision tree','randomforest','boosting','stacking')
    names =('Decision tree', 'SVM', 'Navie bayes','Bagging','Random forest')
    fig = plt.figure()
    fig.suptitle('Algorithm Comparison')
    
   # ax = fig.add_subplot(111)
#    plt.boxplot(results)
    #plt.boxplot(results, labels=names, showmeans = True)
    plt.boxplot(results, labels=names)
    plt.ylabel('Accuracy')    

    #ax.set_xticklabels(names)
    plt.show()            
    
    print ("Results")
    print ("decision tree")
    print (np.mean(acc_DT))    ## gia tri trung binh  
    print (np.std(acc_DT))     ## Do lech chuan   
    
    
    print ("SVM")
    print (np.mean(acc_SVM))    ## gia tri trung binh  
    print (np.std(acc_SVM))     ## Do lech chuan   
    
    print ("Naive Bayes")
    print (np.mean(acc_Navie))    ## gia tri trung binh  
    print (np.std(acc_Navie))     ## Do lech chuan   
    
    print ("Bagging")
    print (np.mean(acc_Bagging))    ## gia tri trung binh  
    print (np.std(acc_Bagging))     ## Do lech chuan   
    
    print ("Random forest")
    print (np.mean(acc_RF))    ## gia tri trung binh  
    print (np.std(acc_RF))     ## Do lech chuan   
    
    print ("Time")
    print (np.mean(time_DT))
    print (np.mean(time_SVM))
    print (np.mean(time_Naive))
    print (np.mean(time_Bagging))
    print (np.mean(time_RF))
    
    
'''
    CNS1  ::  data ok
    Colon  :  data ok 
    Brain   : data ok
    Prostate: data ok 
    LungCancer: data ok
    Adenocarcinoma
    Leukemia4c  
'''  

In [ ]:
import numpy as np